# Linear Algebra

[LinearAlgebra section of the Julia documentation](https://docs.julialang.org/en/v1.0.0/stdlib/LinearAlgebra/)

After `using LinearAlgebra`, Julia speaks linear algebra fluently.

Linear algebra functions in Julia are largely implemented by calling functions from [BLAS](http://www.netlib.org/blas/)/[LAPACK](http://www.netlib.org/lapack/). Sparse factorizations call functions from [SuiteSparse](http://faculty.cse.tamu.edu/davis/suitesparse.html).

As per default, Julia uses the [OpenBLAS](https://github.com/xianyi/OpenBLAS) implementation (BLAS, LAPACK, LIBM), which can be replaced by [Intel's MKL](https://software.intel.com/en-us/mkl) (BLAS, LAPACK) and [Intel's Math Library](https://software.intel.com/en-us/node/522653) (LIBM).

What is all this stuff?!?

* **BLAS**: a collection of low-level matrix and vector arithmetic operations ("multiply two matrices", "multiply a matrix by vector").
* **LAPACK**:  a collection of higher-level linear algebra operations. Things like matrix factorizations (LU, LLt, QR, SVD, Schur, etc) that are used to do things like “find the eigenvalues of a matrix”, or “find the singular values of a matrix”, or “solve a linear system”.
* **LIBM**: basic math functions like `sin`, `cos`, `sinh`, etcetera

Sparse matrices are more difficult and there exist different collections of routines, one of which is **SuiteSparse**.

In [20]:
using LinearAlgebra

In [8]:
A = rand(4,4)

4×4 Array{Float64,2}:
 0.308571  0.55441    0.378221   0.877523 
 0.357562  0.798234   0.0787201  0.207697 
 0.869954  0.0657306  0.313206   0.0713177
 0.986541  0.814281   0.441768   0.26346  

In [9]:
typeof(A)

Array{Float64,2}

In [10]:
Array{Float64, 2} === Matrix{Float64} # equivalent not just equal

true

In [18]:
inv(A)

4×4 Array{Float64,2}:
 -0.0474199   2.43409    3.12083  -2.60576
 -0.336579    0.342629  -1.42338   1.23626
 -0.121251   -7.30558   -5.69018   7.70349
  1.42115     2.07639    2.2544   -3.18505

In [21]:
det(A)

0.0635245935265464

In [22]:
rank(A)

4

Taking linear algebra seriously (see for example this [great video](https://www.youtube.com/watch?v=C2RO34b_oPM) or [issue](https://github.com/JuliaLang/julia/issues/4774), [issue](https://github.com/JuliaLang/julia/issues/20978)).

In [36]:
A + 3

MethodError: MethodError: no method matching +(::Array{Float64,2}, ::Int64)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  +(!Matched::Complex{Bool}, ::Real) at complex.jl:292
  +(!Matched::Missing, ::Number) at missing.jl:93
  ...

In [38]:
A + 3I

4×4 Array{Float64,2}:
 3.30857   0.55441    0.378221   0.877523 
 0.357562  3.79823    0.0787201  0.207697 
 0.869954  0.0657306  3.31321    0.0713177
 0.986541  0.814281   0.441768   3.26346  

In [39]:
A .+ 3

4×4 Array{Float64,2}:
 3.30857  3.55441  3.37822  3.87752
 3.35756  3.79823  3.07872  3.2077 
 3.86995  3.06573  3.31321  3.07132
 3.98654  3.81428  3.44177  3.26346

Let's get a vector as well

In [11]:
v = rand(4)

4-element Array{Float64,1}:
 0.4826073972459104
 0.9373119807156252
 0.9994912136681733
 0.9156837953177845

In [12]:
typeof(v)

Array{Float64,1}

In [13]:
Array{Float64,1} === Vector{Float64}

true

In [23]:
norm(v)

1.7172428914620612

In [41]:
v^2

MethodError: MethodError: no method matching ^(::Array{Float64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:782
  ^(!Matched::Missing, ::Integer) at missing.jl:120
  ^(!Matched::Missing, ::Number) at missing.jl:93
  ...

In [58]:
v.^2

4-element Array{Float64,1}:
 0.23290989987647195
 0.8785537491930485 
 0.9989826861998781 
 0.8384768130075824 

Some things might be suprising

In [44]:
1/v

1×4 Transpose{Float64,Array{Float64,1}}:
 0.163655  0.317849  0.338934  0.310515

But if it works, there is typically meaning to it. In this case it is calculating the [Moore-Penrose-Pseudoinverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse#Vectors) (`transpose(v)/sum(abs2,v)`).

# Solving a linear system `Ax = b`

In [138]:
A = rand(1:10, 3, 3)

3×3 Array{Int64,2}:
  1   3  4
 10  10  9
  6   1  8

In [141]:
b = rand(3)

3-element Array{Float64,1}:
 0.9094533679558146 
 0.21370769840394388
 0.22126613797411077

Solve explicitly

In [147]:
inv(A)*b

3-element Array{Float64,1}:
 -0.2773940838798093 
  0.09761285248365623
  0.22350222359616376

Solve by left division `\`

In [146]:
A\b

3-element Array{Float64,1}:
 -0.27739408387980924
  0.0976128524836564 
  0.22350222359616365

In [150]:
@btime inv($A)*$b;
@btime $A\$b;

  671.140 ns (7 allocations: 2.25 KiB)
  229.867 ns (4 allocations: 416 bytes)


What does Julia do to make this so much faster? It know that it can perform the division much faster if it `LU` decomposes `A`.

In [162]:
lu(A)\b

3-element Array{Float64,1}:
 -0.27739408387980924
  0.0976128524836564 
  0.22350222359616365

In [163]:
@btime lu($A)\$b

  208.903 ns (4 allocations: 416 bytes)


3-element Array{Float64,1}:
 -0.27739408387980924
  0.0976128524836564 
  0.22350222359616365

In [199]:
x = lu(A)\b
y = A\b
maximum(abs.(x-y)) # check that this is what Julia uses internally

0.0

Let's inspect the output of `lu(A)`

In [164]:
lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
  1.0  0.0   0.0
 10.0  1.0   0.0
  6.0  0.85  1.0
U factor:
3×3 Array{Float64,2}:
 1.0    3.0    4.0 
 0.0  -20.0  -31.0 
 0.0    0.0   10.35

In [156]:
typeof(lu(A))

LU{Float64,Array{Float64,2}}

In [159]:
methodswith(LU)

[1] /(B::AbstractArray{T,2} where T, A::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:626
[2] adjoint(F::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:31
[3] iterate(S::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:26
[4] iterate(S::LU, ::Val{:U}) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:27
[5] iterate(S::LU, ::Val{:p}) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:28
[6] iterate(S::LU, ::Val{:done}) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:29
[7] propertynames(F::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:301
[8] propertynames(F::LU, private::Bool) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:301
[9] show(io::IO, mime::MIME{Symbol("text/plain")}, F::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:307
[10] size(A::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:272
[11] size(A::LU, i) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:273
[12] transpose(F::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:32
[13] det(F::LU{T,S} where S<:AbstractArray{T,2}) where T in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:378
[14] issuccess(F::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:304
[15] logabsdet(F::LU{T,S} where S<:AbstractArray{T,2}) where T in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:393
[16] lu(S::LU) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\lu.jl:255

In [161]:
supertype(LU)

Factorization

In [160]:
methodswith(Factorization)

[1] ==(F::T, G::T) where T<:Factorization in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:53
[2] \(F::Factorization, D::Diagonal) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\diagonal.jl:375
[3] \(F::Factorization, B::Union{AbstractArray{T,2}, AbstractArray{T,1}} where T) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:83
[4] convert(::Type{T}, f::T) where T<:Factorization in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:43
[5] convert(::Type{T}, f::Factorization) where T<:Factorization in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:44
[6] convert(::Type{T}, f::Factorization) where T<:AbstractArray in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:46
[7] hash(F::Factorization, h::UInt64) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:52
[8] inv(F::Factorization{T}) where T in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:50
[9] isequal(F::T, G::T) where T<:Factorization in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:54
[10] det(F::Factorization) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:39
[11] ldiv!(Y::Union{AbstractArray{T,2}, AbstractArray{T,1}} where T, A::Factorization, B::Union{AbstractArray{T,2}, AbstractArray{T,1}} where T) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:100
[12] logdet(F::Factorization) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\factorization.jl:34

# Factorizations

### Matrix factorizations

[Matrix factorizations (a.k.a. matrix decompositions)](https://en.wikipedia.org/wiki/Matrix_decomposition)
compute the factorization of a matrix into a product of matrices, and are one of the central concepts
in linear algebra.

The following table summarizes the types of matrix factorizations that have been implemented in
Julia. Details of their associated methods can be found in the [Standard Functions](@ref) section
of the Linear Algebra documentation.

| Type              | Description                                                                                                    |
|:----------------- |:-------------------------------------------------------------------------------------------------------------- |
| `Cholesky`        | [Cholesky factorization](https://en.wikipedia.org/wiki/Cholesky_decomposition)                                 |
| `CholeskyPivoted` | [Pivoted](https://en.wikipedia.org/wiki/Pivot_element) Cholesky factorization                                  |
| `LU`              | [LU factorization](https://en.wikipedia.org/wiki/LU_decomposition)                                             |
| `LUTridiagonal`   | LU factorization for [`Tridiagonal`](@ref) matrices                                                            |
| `QR`              | [QR factorization](https://en.wikipedia.org/wiki/QR_decomposition)                                             |
| `QRCompactWY`     | Compact WY form of the QR factorization                                                                        |
| `QRPivoted`       | Pivoted [QR factorization](https://en.wikipedia.org/wiki/QR_decomposition)                                     |
| `Hessenberg`      | [Hessenberg decomposition](http://mathworld.wolfram.com/HessenbergDecomposition.html)                          |
| `Eigen`           | [Spectral decomposition](https://en.wikipedia.org/wiki/Eigendecomposition_(matrix))                            |
| `SVD`             | [Singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)                     |
| `GeneralizedSVD`  | [Generalized SVD](https://en.wikipedia.org/wiki/Generalized_singular_value_decomposition#Higher_order_version) |

Taken from: https://docs.julialang.org/en/v1.0.0/stdlib/LinearAlgebra/#man-linalg-factorizations-1

In [173]:
@btime lu($A)\$b
@btime qr($A)\$b
@btime svd($A)\$b;

  207.591 ns (4 allocations: 416 bytes)
  2.095 μs (9 allocations: 976 bytes)
  4.328 μs (18 allocations: 3.30 KiB)


In [174]:
?\

search: \



```
\(x, y)
```

Left division operator: multiplication of `y` by the inverse of `x` on the left. Gives floating-point results for integer arguments.

# Examples

```jldoctest
julia> 3 \ 6
2.0

julia> inv(3) * 6
2.0

julia> A = [1 2; 3 4]; x = [5, 6];

julia> A \ x
2-element Array{Float64,1}:
 -4.0
  4.5

julia> inv(A) * x
2-element Array{Float64,1}:
 -4.0
  4.5
```

---

```
\(A, B)
```

Matrix division using a polyalgorithm. For input matrices `A` and `B`, the result `X` is such that `A*X == B` when `A` is square. The solver that is used depends upon the structure of `A`.  If `A` is upper or lower triangular (or diagonal), no factorization of `A` is required and the system is solved with either forward or backward substitution. For non-triangular square matrices, an LU factorization is used.

For rectangular `A` the result is the minimum-norm least squares solution computed by a pivoted QR factorization of `A` and a rank estimate of `A` based on the R factor.

When `A` is sparse, a similar polyalgorithm is used. For indefinite matrices, the `LDLt` factorization does not use pivoting during the numerical factorization and therefore the procedure can fail even for invertible matrices.

# Examples

```jldoctest
julia> A = [1 0; 1 -2]; B = [32; -4];

julia> X = A \ B
2-element Array{Float64,1}:
 32.0
 18.0

julia> A * X == B
true
```

---

```
(\)(F::QRSparse, B::StridedVecOrMat)
```

Solve the least squares problem $\min\|Ax - b\|^2$ or the linear system of equations $Ax=b$ when `F` is the sparse QR factorization of $A$. A basic solution is returned when the problem is underdetermined.

# Examples

```jldoctest
julia> A = sparse([1,2,4], [1,1,1], [1.0,1.0,1.0], 4, 2)
4×2 SparseMatrixCSC{Float64,Int64} with 3 stored entries:
  [1, 1]  =  1.0
  [2, 1]  =  1.0
  [4, 1]  =  1.0

julia> qr(A)\fill(1.0, 4)
2-element Array{Float64,1}:
 1.0
 0.0
```


This is what the actual heuristic looks like (`@which`/`@edit` are your friends!)

```julia
function (\)(A::AbstractMatrix, B::AbstractVecOrMat)
    @assert !has_offset_axes(A, B)
    m, n = size(A)
    if m == n
        if istril(A)
            if istriu(A)
                return Diagonal(A) \ B
            else
                return LowerTriangular(A) \ B
            end
        end
        if istriu(A)
            return UpperTriangular(A) \ B
        end
        return lu(A) \ B
    end
    return qr(A,Val(true)) \ B
end
```

### Choosing a factorization

In [175]:
?factorize

search: factorize Factorization factorial



```
factorize(A)
```

Compute a convenient factorization of `A`, based upon the type of the input matrix. `factorize` checks `A` to see if it is symmetric/triangular/etc. if `A` is passed as a generic matrix. `factorize` checks every element of `A` to verify/rule out each property. It will short-circuit as soon as it can rule out symmetry/triangular structure. The return value can be reused for efficient solving of multiple systems. For example: `A=factorize(A); x=A\b; y=A\C`.

| Properties of `A`          | type of factorization                      |
|:-------------------------- |:------------------------------------------ |
| Positive-definite          | Cholesky (see [`cholesky`](@ref))          |
| Dense Symmetric/Hermitian  | Bunch-Kaufman (see [`bunchkaufman`](@ref)) |
| Sparse Symmetric/Hermitian | LDLt (see [`ldlt`](@ref))                  |
| Triangular                 | Triangular                                 |
| Diagonal                   | Diagonal                                   |
| Bidiagonal                 | Bidiagonal                                 |
| Tridiagonal                | LU (see [`lu`](@ref))                      |
| Symmetric real tridiagonal | LDLt (see [`ldlt`](@ref))                  |
| General square             | LU (see [`lu`](@ref))                      |
| General non-square         | QR (see [`qr`](@ref))                      |

If `factorize` is called on a Hermitian positive-definite matrix, for instance, then `factorize` will return a Cholesky factorization.

# Examples

```jldoctest
julia> A = Array(Bidiagonal(fill(1.0, (5, 5)), :U))
5×5 Array{Float64,2}:
 1.0  1.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0
 0.0  0.0  1.0  1.0  0.0
 0.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0

julia> factorize(A) # factorize will check to see that A is already factorized
5×5 Bidiagonal{Float64,Array{Float64,1}}:
 1.0  1.0   ⋅    ⋅    ⋅
  ⋅   1.0  1.0   ⋅    ⋅
  ⋅    ⋅   1.0  1.0   ⋅
  ⋅    ⋅    ⋅   1.0  1.0
  ⋅    ⋅    ⋅    ⋅   1.0
```

This returns a `5×5 Bidiagonal{Float64}`, which can now be passed to other linear algebra functions (e.g. eigensolvers) which will use specialized methods for `Bidiagonal` types.


In [201]:
typeof(factorize(A))

LU{Float64,Array{Float64,2}}

In [202]:
typeof(factorize(A+A'))

BunchKaufman{Float64,Array{Float64,2}}

# Use types to tag your matrices

Ok, we've seen that Julia analyses the input matrix using some heuristic, factorizes it appropriately to then perform the calculation efficiently. 

But we can (and probably should) also be more explicit about our input to avoid this heuristic. We can encode the special structure of our matrix in a type such that we dispatch directly to the efficient method. Remember, the types decide which method is actually being run!

There are many reasons to indicate what kind of matrix we have.

* Don't rely on a heuristic. Not all methods have one!
* The heurisitc comes with a small performance penalty.
* The heurisitc isn't perfect and might fail to notice our matrix's special structure. Maybe because it's not known to base Julia. As we'll see later on, many external packages define additional special matrix types and efficient procedures for them.

The following special matrix types are available out-of-the-box.

### Special matrices

[Matrices with special symmetries and structures](http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=3274)
arise often in linear algebra and are frequently associated with various matrix factorizations.
Julia features a rich collection of special matrix types, which allow for fast computation with
specialized routines that are specially developed for particular matrix types.

The following tables summarize the types of special matrices that have been implemented in Julia,
as well as whether hooks to various optimized methods for them in LAPACK are available.

| Type                      | Description                                                                      |
|:------------------------- |:-------------------------------------------------------------------------------- |
| [`Symmetric`](@ref)       | [Symmetric matrix](https://en.wikipedia.org/wiki/Symmetric_matrix)               |
| [`Hermitian`](@ref)       | [Hermitian matrix](https://en.wikipedia.org/wiki/Hermitian_matrix)               |
| [`UpperTriangular`](@ref) | Upper [triangular matrix](https://en.wikipedia.org/wiki/Triangular_matrix)       |
| [`LowerTriangular`](@ref) | Lower [triangular matrix](https://en.wikipedia.org/wiki/Triangular_matrix)       |
| [`Tridiagonal`](@ref)     | [Tridiagonal matrix](https://en.wikipedia.org/wiki/Tridiagonal_matrix)           |
| [`SymTridiagonal`](@ref)  | Symmetric tridiagonal matrix                                                     |
| [`Bidiagonal`](@ref)      | Upper/lower [bidiagonal matrix](https://en.wikipedia.org/wiki/Bidiagonal_matrix) |
| [`Diagonal`](@ref)        | [Diagonal matrix](https://en.wikipedia.org/wiki/Diagonal_matrix)                 |
| [`UniformScaling`](@ref)  | [Uniform scaling operator](https://en.wikipedia.org/wiki/Uniform_scaling)        |

See: https://docs.julialang.org/en/v1.0.0/stdlib/LinearAlgebra/#Special-matrices-1

In [251]:
D = Diagonal(1:3)

3×3 Diagonal{Int64,UnitRange{Int64}}:
 1  ⋅  ⋅
 ⋅  2  ⋅
 ⋅  ⋅  3

In [252]:
Ddense = Matrix(D) # same matrix but type doesn't indicate diagonal structure

3×3 Array{Int64,2}:
 1  0  0
 0  2  0
 0  0  3

In [253]:
X = rand(1:10, 3,3) # truly dense matrix for comparison

3×3 Array{Int64,2}:
 9  5   5
 8  3   2
 6  9  10

In [254]:
@btime $D*$b
@btime $Ddense*$b
@btime $X*$b

  29.062 ns (1 allocation: 112 bytes)
  39.582 ns (1 allocation: 112 bytes)
  39.583 ns (1 allocation: 112 bytes)


3-element Array{Float64,1}:
 10.359949493492604
  8.35928231480657 
  9.592750873111491

What does it actually dispatch to?

In [243]:
@which D*b

*(D::Diagonal, V::AbstractArray{T,1} where T) in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\diagonal.jl:159

In [244]:
@which Ddense*b

*(A::AbstractArray{T,2}, x::AbstractArray{S,1}) where {T, S} in LinearAlgebra at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\matmul.jl:49

**Dense Diagonal** (`Ddense*b`)

```julia
function (*)(A::AbstractMatrix{T}, x::AbstractVector{S}) where {T,S}
    TS = promote_op(matprod, T, S)
    mul!(similar(x,TS,axes(A,1)),A,x)
end
```

**Diagonal** (`D*b`)
```julia
(*)(D::Diagonal, V::AbstractVector) = D.diag .* V
```

# Hamiltonian-like matrices

Let's build a simple Hamiltonian-like matrix

In [272]:
N = 100
μ = fill(-0.5, N)
t = fill(1, N-1);

In [273]:
H = diagm(0 => μ, 1 => t, -1 => t)

100×100 Array{Float64,2}:
 -0.5   1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  1.0  -0.5   1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   1.0  -0.5   1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   1.0  -0.5   1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   1.0  -0.5   1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   1.0  -0.5  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0

In [274]:
function transform(H)
    # Some operation on H
    return H*H
end

transform (generic function with 1 method)

In [275]:
@btime transform($H);

  37.410 μs (2 allocations: 78.20 KiB)


In [276]:
typeof(H)

Array{Float64,2}

As long as the code is generic (respects the informal `AbstractArray` interface), we can use the same piece of code for completely different array types.

In [277]:
Hsparse = sparse(H)

100×100 SparseMatrixCSC{Float64,Int64} with 298 stored entries:
  [1  ,   1]  =  -0.5
  [2  ,   1]  =  1.0
  [1  ,   2]  =  1.0
  [2  ,   2]  =  -0.5
  [3  ,   2]  =  1.0
  [2  ,   3]  =  1.0
  [3  ,   3]  =  -0.5
  [4  ,   3]  =  1.0
  [3  ,   4]  =  1.0
  [4  ,   4]  =  -0.5
  [5  ,   4]  =  1.0
  [4  ,   5]  =  1.0
  ⋮
  [97 ,  96]  =  1.0
  [96 ,  97]  =  1.0
  [97 ,  97]  =  -0.5
  [98 ,  97]  =  1.0
  [97 ,  98]  =  1.0
  [98 ,  98]  =  -0.5
  [99 ,  98]  =  1.0
  [98 ,  99]  =  1.0
  [99 ,  99]  =  -0.5
  [100,  99]  =  1.0
  [99 , 100]  =  1.0
  [100, 100]  =  -0.5

In [278]:
@btime transform($Hsparse);

  19.169 μs (309 allocations: 32.11 KiB)


Let's exploit the banded structure of the Hamiltonian

In [279]:
using BandedMatrices
Hbanded = BandedMatrix(0 => μ, 1 => t, -1 => t)

100×100 BandedMatrix{Float64,Array{Float64,2}}:
 -0.5   1.0    ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  1.0  -0.5   1.0    ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅    1.0  -0.5   1.0    ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅    1.0  -0.5   1.0    ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅    1.0  -0.5   1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅    1.0  -0.5  …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅    1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅   

In [280]:
@btime transform($Hbanded);

  4.682 μs (2 allocations: 4.11 KiB)


## It's not that easy though

Unfortunately life's not that simple. Choosing the best type (and therewith an algorithm) can be tricky and one has to play around a bit.

For example, you would expect that `Tridiagonal` is a reasonable type for aboves example. Turns out that multiplication of two `Tridiagonal` matrices is slower than the generic matrix-matrix product.

In [283]:
Htr = Tridiagonal(H)

100×100 Tridiagonal{Float64,Array{Float64,1}}:
 -0.5   1.0    ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  1.0  -0.5   1.0    ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅    1.0  -0.5   1.0    ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅    1.0  -0.5   1.0    ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅    1.0  -0.5   1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅    1.0  -0.5  …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅    1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅    

In [285]:
@btime transform($H);
@btime transform($Htr);

  33.082 μs (2 allocations: 78.20 KiB)
  48.850 μs (6 allocations: 234.61 KiB)


The good thing is that it's very easy to try out different types!

# Move everything to the GPU

Just to push this idea of "generic coding" a bit more, want to put the calculation on a GPU? Sure... (if you have one :D)

In [291]:
using GPUArrays

In [292]:
# Hgpu = CuArray(H) # CUDA, CuArrays.jl
# Hgpu = CLArray(H) # Open-CL, CLArrays.jl
Hgpu = JLArray(H) # I don't have a GPU so I'll take a fake GPU Array on the CPU :)

100×100 JLArray{Float64,2}:
 -0.5   1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  1.0  -0.5   1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   1.0  -0.5   1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   1.0  -0.5   1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   1.0  -0.5   1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   1.0  -0.5  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0

In [293]:
@btime transform($Hgpu); # of course it's no faster without a real GPU :)

  37.720 μs (3 allocations: 78.23 KiB)


With `DistributedArray` we'll learn about another type that is similar in spirit.

# Eigendecomposition

Note that there are some solvers for dense matrices, others for sparse matrices, some "classical" ones, and other iterative solvers.

Since Julia is currently mostly calling Fortran libraries (LAPACK or ARPACK.jl for sparse matrices) you won't get any real performance benefit here.

## Dense matrices

In [294]:
eigen(H)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
100-element Array{Float64,1}:
 -2.4990325645839753
 -2.4961311942671887
 -2.4912986959380365
 -2.484539744726553 
 -2.475860879481513 
 -2.4652704964445267
 -2.4527788411272136
 -2.438397998399332 
 -2.422141880797449 
 -2.4040262150654583
 -2.384068526939977 
 -2.3622881241953175
 -2.3387060779644715
  ⋮                 
  1.362288124195319 
  1.3840685269399784
  1.4040262150654599
  1.422141880797449 
  1.4383979983993322
  1.452778841127214 
  1.4652704964445273
  1.4758608794815133
  1.484539744726553 
  1.4912986959380372
  1.4961311942671887
  1.4990325645839762
eigenvectors:
100×100 Array{Float64,2}:
  0.00437636   0.00874848  -0.0131121  …  0.0131121   0.00874848  0.00437636
 -0.00874848  -0.0174631    0.0261102     0.0261102   0.0174631   0.00874848
  0.0131121    0.0261102   -0.038881      0.038881    0.0261102   0.0131121 
 -0.0174631   -0.0346562    0.0513136     0.0513136   0.0346562   0.0174631 
  0.02

In [305]:
eigvals(H)

100-element Array{Float64,1}:
 -2.4990325645839757
 -2.496131194267189 
 -2.4912986959380365
 -2.4845397447265527
 -2.475860879481514 
 -2.4652704964445276
 -2.4527788411272144
 -2.4383979983993314
 -2.422141880797449 
 -2.4040262150654588
 -2.384068526939979 
 -2.3622881241953184
 -2.338706077964473 
  ⋮                 
  1.3622881241953189
  1.3840685269399777
  1.4040262150654603
  1.4221418807974493
  1.438397998399332 
  1.4527788411272136
  1.4652704964445276
  1.4758608794815136
  1.4845397447265527
  1.491298695938037 
  1.4961311942671893
  1.499032564583976 

In [304]:
@btime eigen($H);
@btime eigvals($H);

  1.786 ms (15 allocations: 272.13 KiB)
  1.338 ms (12 allocations: 115.84 KiB)


Since Julia is using eigenproblem solvers from LAPACK (written in a low-level language) the code is, of course, not generic.

The best Julia can do is manually dispatch to the best LAPACK routine available.

Hence, it won't work with most of our special matrices.

In [330]:
eigen(Diagonal(H)); # works (different Hamiltonian)

In [328]:
eigen(Tridiagonal(H)); # should probably either fallback to a generic routine or error

MethodError: MethodError: no method matching eigen(::Tridiagonal{Float64,Array{Float64,1}})
Closest candidates are:
  eigen(::AbstractArray{TA,2}, !Matched::AbstractArray{TB,2}) where {TA, TB} at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\eigen.jl:382
  eigen(!Matched::SymTridiagonal{T,V} where V<:AbstractArray{T,1}) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\tridiag.jl:201
  eigen(!Matched::SymTridiagonal{T,V} where V<:AbstractArray{T,1}, !Matched::UnitRange) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\tridiag.jl:205
  ...

In [327]:
# doesn't work because BandedMatrices.jl is an external package (without any eigen solvers) that is unkown to base Julia
eigen(Hbanded);

MethodError: MethodError: no method matching eigen(::BandedMatrix{Float64,Array{Float64,2}})
Closest candidates are:
  eigen(::AbstractArray{TA,2}, !Matched::AbstractArray{TB,2}) where {TA, TB} at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\eigen.jl:382
  eigen(!Matched::SymTridiagonal{T,V} where V<:AbstractArray{T,1}) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\tridiag.jl:201
  eigen(!Matched::SymTridiagonal{T,V} where V<:AbstractArray{T,1}, !Matched::UnitRange) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\tridiag.jl:205
  ...

In [326]:
eigen(Symmetric(H));

In [325]:
@btime eigen($(Symmetric(H))); # no faster than using plain H

  1.720 ms (15 allocations: 272.13 KiB)


And most importantly, the sparse case

In [332]:
eigen(Hsparse)

ErrorException: eigen(A) not supported for sparse matrices. Use for example eigs(A) from the Arpack package instead.

Let's follow Julia's advice and take a look at [ARPACK.jl](https://github.com/JuliaLinearAlgebra/Arpack.jl).

## Sparse -  Arpack.jl

Wrapper to Fortran library [ARPACK](https://www.caam.rice.edu/software/ARPACK/) which implements **iterative** eigenvalue and singular value solvers.

This is the same library that scipy uses.

In [336]:
using Arpack

In [337]:
?eigs # because it's a wrapper the function has a different name here

search: eigs eigvecs eigvals eigvals! leading_ones leading_zeros eigen eigmin



```
eigs(A; nev=6, ncv=max(20,2*nev+1), which=:LM, tol=0.0, maxiter=300, sigma=nothing, ritzvec=true, v0=zeros((0,))) -> (d,[v,],nconv,niter,nmult,resid)
```

Computes eigenvalues `d` of `A` using implicitly restarted Lanczos or Arnoldi iterations for real symmetric or general nonsymmetric matrices respectively. See [the manual](@ref lib-itereigen) for more information.

`eigs` returns the `nev` requested eigenvalues in `d`, the corresponding Ritz vectors `v` (only if `ritzvec=true`), the number of converged eigenvalues `nconv`, the number of iterations `niter` and the number of matrix vector multiplications `nmult`, as well as the final residual vector `resid`.

# Examples

```jldoctest
julia> using Arpack

julia> A = Diagonal(1:4);

julia> λ, ϕ = eigs(A, nev = 2);

julia> λ
2-element Array{Float64,1}:
 4.0
 3.0
```

---

```
eigs(A, B; nev=6, ncv=max(20,2*nev+1), which=:LM, tol=0.0, maxiter=300, sigma=nothing, ritzvec=true, v0=zeros((0,))) -> (d,[v,],nconv,niter,nmult,resid)
```

Computes generalized eigenvalues `d` of `A` and `B` using implicitly restarted Lanczos or Arnoldi iterations for real symmetric or general nonsymmetric matrices respectively. See [the manual](@ref lib-itereigen) for more information.


In [371]:
λ, ϕ = eigs(Hsparse)

([-2.49903, -2.49613, -2.4913, -2.48454, -2.47586, -2.46527], [0.00437636 -0.00874848 … 0.0217972 0.0261102; -0.00874848 0.0174631 … -0.0430682 -0.0513136; … ; 0.00874848 0.0174631 … 0.0430682 -0.0513136; -0.00437636 -0.00874848 … -0.0217972 0.0261102], 6, 26, 344, [-0.157345, 0.207757, -0.0903062, -0.0448015, -0.060786, 0.139359, 0.106548, 0.239218, 0.0561659, 0.12447  …  0.13641, 0.108069, 0.326383, -0.108833, 0.0531306, 0.0265787, 0.0399327, -0.0496378, 0.136597, 0.0123446])

In [374]:
λ

6-element Array{Float64,1}:
 -2.499032564583982 
 -2.4961311942671998
 -2.491298695938051 
 -2.484539744726561 
 -2.475860879481513 
 -2.4652704964445307

In [375]:
ϕ

100×6 Array{Float64,2}:
  0.00437636  -0.00874848   0.0131121  -0.0174631   0.0217972   0.0261102
 -0.00874848   0.0174631   -0.0261102   0.0346562  -0.0430682  -0.0513136
  0.0131121   -0.0261102    0.038881   -0.0513136   0.0632996   0.0747349
 -0.0174631    0.0346562   -0.0513136   0.0671776  -0.082003   -0.0955607
  0.0217972   -0.0430682    0.0632996  -0.082003    0.098727    0.113068 
 -0.0261102    0.0513136   -0.0747349   0.0955607  -0.113068   -0.126648 
  0.0303979   -0.0593604    0.0855198  -0.107641    0.124679    0.13583  
 -0.0346562    0.0671776   -0.0955607   0.118057   -0.133281   -0.140294 
  0.038881    -0.0747349    0.10477    -0.126648    0.138665    0.139886 
 -0.0430682    0.082003    -0.113068    0.133281   -0.140702   -0.13462  
  0.0472137   -0.0889539    0.120382   -0.137853    0.139343    0.124679 
 -0.0513136    0.0955607   -0.126648    0.140294   -0.13462    -0.110408 
  0.0553638   -0.101798     0.131812   -0.140566    0.126648    0.092302 
  ⋮           

In [368]:
@btime eigen($H);
@btime eigs($H);
@btime eigs($Hsparse); # it's slower because our Hsparse is actually pretty small and non-sparse

  1.892 ms (15 allocations: 272.13 KiB)
  7.104 ms (1279 allocations: 78.55 KiB)
  2.314 ms (1268 allocations: 78.95 KiB)


In [364]:
length(Hsparse.nzval) / length(Hsparse)

0.0298

In [365]:
size(Hsparse)

(100, 100)

The future might be the pure Julia packages [SparseLinearAlgebra.jl](https://github.com/JuliaLinearAlgebra/SparseLinearAlgebra.jl) and [IterativeSolvers.jl](https://github.com/JuliaMath/IterativeSolvers.jl).

The latter are so called matrix free algorithms. Let's learn from an example what this actually means.

## Linear operators / maps

See https://github.com/Jutho/LinearMaps.jl

In [104]:
using LinearMaps

In [117]:
?LinearMap

search: LinearMap LinearMaps LinearImplicitEuler



```
LinearMap(A; kwargs...)
LinearMap{T=Float64}(f, [fc,], M::Int, N::Int = M; kwargs...)
```

Construct a linear map object, either from an existing `LinearMap` or `AbstractMatrix` `A`, with the purpose of redefining its properties via the keyword arguments `kwargs`, or from a function or callable object `f`. In the latter case, one also needs to specify the size of the equivalent matrix representation `(M, N)`, i.e. for functions `f` acting on length `N` vectors and producing length `M` vectors (with default value `N=M`). Preferably, also the `eltype` `T` of the corresponding matrix representation needs to be specified, i.e. whether the action of `f` on a vector will be similar to e.g. multiplying by numbers of type `T`. If not specified, the devault value `T=Float64` will be assumed. Optionally, a corresponding function `fc` can be specified that implements the transpose/adjoint of `f`.

The keyword arguments and their default values for functions `f` are

  * issymmetric::Bool = false : whether `A` or `f` acts as a symmetric matrix
  * ishermitian::Bool = issymmetric & T<:Real : whether `A` or `f` acts as a Hermitian matrix
  * isposdef::Bool = false : whether `A` or `f` acts as a positive definite matrix.

For existing linear maps or matrices `A`, the default values will be taken by calling `issymmetric`, `ishermitian` and `isposdef` on the existing object `A`.

For functions `f`, there is one more keyword arguments

  * ismutating::Bool : flags whether the function acts as a mutating matrix multiplication   `f(y,x)` where the result vector `y` is the first argument (in case of `true`),   or as a normal matrix multiplication that is called as `y=f(x)` (in case of `false`).   The default value is guessed by looking at the number of arguments of the first occurence   of `f` in the method table.


In [141]:
H = diagm(0 => 1:1000) # diagonal but not properly typed

1000×1000 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0   0   0   0  …    0    0    0    0    0    0     0
 0  2  0  0  0  0  0  0  0   0   0   0       0    0    0    0    0    0     0
 0  0  3  0  0  0  0  0  0   0   0   0       0    0    0    0    0    0     0
 0  0  0  4  0  0  0  0  0   0   0   0       0    0    0    0    0    0     0
 0  0  0  0  5  0  0  0  0   0   0   0       0    0    0    0    0    0     0
 0  0  0  0  0  6  0  0  0   0   0   0  …    0    0    0    0    0    0     0
 0  0  0  0  0  0  7  0  0   0   0   0       0    0    0    0    0    0     0
 0  0  0  0  0  0  0  8  0   0   0   0       0    0    0    0    0    0     0
 0  0  0  0  0  0  0  0  9   0   0   0       0    0    0    0    0    0     0
 0  0  0  0  0  0  0  0  0  10   0   0       0    0    0    0    0    0     0
 0  0  0  0  0  0  0  0  0   0  11   0  …    0    0    0    0    0    0     0
 0  0  0  0  0  0  0  0  0   0   0  12       0    0    0    0    0    0     0
 0  0  0  0  0  0  0  0  0   0   0   0

In [142]:
@btime eigs($H);

  153.608 ms (2046 allocations: 7.94 MiB)


In [159]:
# Define map through matrix-vector multiplication. In-place version, y is the result.
function Hmult!(y::AbstractVector, x::AbstractVector)
    N = length(x)
    @inbounds for i=1:N
        y[i] = i*x[i]
    end
    nothing
end

D = LinearMap(Hmult!, 1000; ismutating=true, issymmetric=true, isposdef=true)

@btime eigs($D);

  40.323 ms (2533 allocations: 334.16 KiB)


However, this is not faster than using `SparseMatrixCSC` or `Diagonal`.

In [152]:
Hsparse = sparse(H);
@btime eigs($Hsparse);

  41.547 ms (2075 allocations: 335.95 KiB)


In [154]:
Hdiag = Diagonal(H);
@btime eigs($Hdiag);

  42.279 ms (2094 allocations: 328.70 KiB)


# Numpy speed comparison

In [408]:
using LinearAlgebra, PyCall, BenchmarkTools
@pyimport numpy as np

In [409]:
X = rand(1000,1000);

# python
rpy = np.linalg[:eig](X);

# julia
r = eigen(X);

# check eigenvalues
maximum(sort(abs.(r.values)) - sort(abs.(rpy[1]))) # maximum abs difference of eigenvalues

2.3447910280083306e-13

In [410]:
# Benchmark
@btime np.linalg[:eig]($X);
@btime eigen($X);

  1.052 s (165 allocations: 15.28 MiB)
  1.016 s (45 allocations: 31.57 MiB)


In [412]:
# Benchmark - symmetrized
X = X + transpose(X) # symmetrize (but keep generic type to make it fair)

@btime eigen($X);
@btime np.linalg[:eig]($X); # seems to be slower because it doesn't seem to check for symmetry

  231.522 ms (15 allocations: 23.25 MiB)
  1.052 s (4004172 allocations: 129.79 MiB)


### What about sparse matrices?

Unfortunately, we need some (tiny) conversion functions.

In [395]:
using PyCall
@pyimport scipy.sparse as pysparse

"""
    sparsejl2py(X)

Convert julia sparse matrix to python (scipy csc) sparse matrix.
"""
function sparsejl2py(S::SparseMatrixCSC)
  pysparse.csc_matrix((S.nzval, S.rowval .- 1, S.colptr .- 1), shape=size(S))
end

"""
    sparsepy2jl(X::PyObject)

Convert python sparse matrix to julia sparse matrix.
"""
sparsepy2jl(S) =
    SparseMatrixCSC(S[:m], S[:n], S[:indptr] .+ 1, S[:indices] .+ 1, S[:data])

sparsepy2jl

In [378]:
X = sprand(100000,100000,1e-5);
Xpy = sparsejl2py(X)

PyObject <100000x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 99688 stored elements in Compressed Sparse Column format>

In [401]:
@pyimport scipy.sparse.linalg as scipy
# python
rpy = scipy.eigs(Xpy)

# julia
r = eigs(X)

(Complex{Float64}[0.687346+0.0im, -4.85723e-16+0.59115im, -4.85723e-16-0.59115im, -0.59115+0.0im, 0.59115+0.0im, -0.590928+0.0im], Complex{Float64}[-2.56971e-20+0.0im 5.20715e-21-5.30628e-21im … 4.02235e-23+0.0im -1.83038e-20+0.0im; 2.21014e-20+0.0im -3.81312e-21+3.96956e-21im … 1.31167e-22+0.0im 1.5037e-20+0.0im; … ; 1.31805e-20+0.0im -2.65787e-21+2.66813e-21im … 9.01171e-24+0.0im 9.3059e-21+0.0im; -9.8744e-19+0.0im 2.04001e-19+2.78387e-19im … -8.6112e-21+0.0im -1.78495e-18+0.0im], 6, 19, 249, [3.22335e-19, -2.0667e-18, 3.19084e-19, -1.3239e-19, 4.37178e-15, -2.38686e-19, 9.16878e-14, -2.25677e-17, -2.2533e-15, -1.0442e-16  …  3.52047e-18, -9.16409e-21, -1.06057e-7, 1.7677e-19, 3.35373e-16, 1.3887e-16, 3.94468e-14, -8.59802e-20, -1.62549e-19, -1.00281e-10])

In [407]:
isapprox(sort(real.(r[1])), sort(real.(rpy[1])))

true

In [393]:
@btime Arpack.eigs($X, nev=6, which=:LM);
@btime scipy.eigs($Xpy, k=6, which=:LM);

  721.323 ms (1106 allocations: 61.85 MiB)
  613.550 ms (180 allocations: 9.16 MiB)
